In [78]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')

from IPython.core.display import Image
import uuid 

import gc

In [79]:
#R call

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
fastclime = importr('fastclime')
grdevices = importr('grDevices')
base = importr('base')
stats = importr('stats')

In [81]:
#Call custom Python module
import parametric as param

In [82]:
def is_Hermitian(m):
    #Set missing to zero prior to checking symmetry
    m[np.isnan(m)] = 0.

    try:
        return np.allclose(np.transpose(m,(1,0)), m)
    except:
        return False

In [83]:
def fastclime_main(x,lambda_min=0.1,nlambda=50):

    cov_input = 1
    SigmaInput = x.copy()
    
    #Check if matrix is symmetric
    if not is_Hermitian(SigmaInput):
        SigmaInput = np.corrcoef(SigmaInput.T)
        cov_input = 0
    
    #Run parametric simplex linear solver
    Sigmahat, mu, maxnlambda, iicov = param.mainfunc(SigmaInput,lambda_min,nlambda)
      
    #Process output
    maxnlambda+=1
    
    #Reshape the array in Fortran order
    #and then slice the array to extract only the top maxnlambda rows
    lambdamtx = mu.T.reshape(nlambda, -1, order='F')[:maxnlambda,:]
    mu = None
    
    #Take each row of iicov and convert it to a d x d matrix
    d = Sigmahat.shape[1]
    icovlist = np.empty((d, d, maxnlambda)) 
    
    for i in range(maxnlambda):
        icovlist[:,:,i] = iicov[:,i].reshape((d,d)).T
    
    gc.collect()
    
    return x, cov_input, Sigmahat, maxnlambda, lambdamtx, icovlist
    

In [84]:
# #Generate data

L = fastclime.fastclime_generator(n = 100, d = 20)
pydat = np.array(L.rx2('data'))

Generating data from the multivariate normal distribution with the random graph structure....done.


In [85]:
Pyout = fastclime_main(pydat,0.1)

In [91]:
Pyout[5][:,:,3][:,3]

array([ 0.    ,  0.    ,  0.    ,  1.1377,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.2847,  0.    ,  0.    ,  0.    ,  0.    ,
        0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ])

In [54]:
#Compare results to R

In [55]:
#%timeit -n1 -r1 fastclime.fastclime(L.rx2('data'),0.1)
# Rout = fastclime.fastclime(L.rx2('data'),0.1)

Allocating memory 
start recovering 
preparing precision and path matrix list 
Done! 


In [58]:
# from random import randint
# ranint = randint(0,Pyout[3]-1)
# print np.isclose(Pyout[3],np.array(Rout.rx2('maxnlambda')))
# print np.allclose(Pyout[4],np.array(Rout.rx2('lambdamtx')))
# print np.allclose(Pyout[5][:,:,ranint],np.array(Rout.rx2('icovlist')[ranint]))

[ True]
True
True


In [ ]:
def fastclime_lambda(lambdamtx,icovlist,lambda_val):
    
    #Define constants and initialize arrays
    d = icovlist[:,:,0].shape[1]
    maxnlambda = lambdamtx.shape[1]
    icov = np.empty((d, d)) 
    path = np.empty((d, d))
    seq = np.empty((1,d),dtype=int)
    threshold = 1e-5
    status = 0
    
    for i in range(d):
        
        seq[i] = sum(lambdamtx[:,i]>lambda_val)
        
        if seq[i]+1>maxnlambda:
            status = 1
            icov[:,1] = icovlist[seq[i]][:,i]
        else:
            icov[:,i] = icovlist[seq[i]+1][:,i]

    icov = (icov+icov.T)/2.
    tmpicov = icov
    np.fill_diagonal(tmpicov,0.)
    path = Matrix(tmpicov > threshold, sparse=TRUE)*1
            
    return 

fastclime.lambda <- function(lambdamtx, icovlist, lambda)
{

  gcinfo(FALSE)
  d<-dim(icovlist[[1]])[2]
  maxnlambda<-dim(lambdamtx)[1]
  icov<-matrix(0,d,d)
  path<-matrix(0,d,d)
  seq<-rep(0,d)
  threshold<-1e-5
  status<-0
  
  for(i in 1:d)
  {
    
      temp_lambda<-which(lambdamtx[,i]>lambda)
      seq[i]<-length(temp_lambda)
      
      if((seq[i]+1)>maxnlambda)
      {
        status<-1
        icov[,i]<-icovlist[[seq[i]]][,i]
      }
      else{
        icov[,i]<-icovlist[[seq[i]+1]][,i]
      }
     
  }
  
  icov<-(icov+t(icov))/2
  tmpicov<-icov
  diag(tmpicov)<-0
  path<-Matrix(tmpicov>threshold, sparse=TRUE)*1
 
  
  sparsity<-(sum(path))/(d^2-d)
  
  if(status==1)
  {
    cat("Some columns do not reach the required lambda!\n You may want to increase lambda.min or use a large nlambda. \n")
  }
  
  rm(temp_lambda,seq,d,threshold)
  gc()
  
  result<-list("icov"=icov, "path"=path,"sparsity"=sparsity)
  class(result)="fastclime.lambda"
  
  return(result)


}


In [93]:
x = np.arange(9).reshape((3,3))
np.diag(x)

array([0, 4, 8])

In [ ]:
fastclime.plot = function(G, epsflag = FALSE, graph.name = "default", cur.num = 1, location=NULL){
	gcinfo(FALSE)
	if(missing(location))	location = getwd()
	setwd(location)
        diag(G)=0
        Matrix(G,sparse=TRUE)
	g = graph.adjacency(as.matrix(G!=0), mode="undirected", diag=FALSE)
	layout.grid = layout.fruchterman.reingold(g)
	
   	if(epsflag == TRUE)	postscript(paste(paste(graph.name, cur.num, sep=""), "eps", sep="."), width = 8.0, height = 8.0)             
	par(mfrow = c(1,1))
	plot(g, layout=layout.grid, edge.color='gray50',vertex.color="red", vertex.size=2, vertex.label=NA)
	rm(g,location)	
   	gc()
   	if(epsflag == TRUE) dev.off()
}

#Adjacency(matrix, mode=ADJ_UNDIRECTED)
#http://sociograph.blogspot.com/2012/11/visualizing-adjacency-matrices-in-python.html
#https://networkx.github.io/documentation/latest/reference/generated/networkx.drawing.layout.spring_layout.html

In [ ]:
#http://igraph.org/python/doc/igraph.Graph-class.html#layout_fruchterman_reingold

In [ ]:
###Draw network graph


#Use igraph
#http://igraph.org/redirect.html#4


# """
# Draw a graph with matplotlib.
# You must have matplotlib for this to work.
# """
# try:
#     import matplotlib.pyplot as plt
# except:
#     raise

# import networkx as nx

# G=nx.path_graph(8)
# nx.draw(G)
# plt.savefig("simple_path.png") # save as png
# plt.show() # display